In [81]:

import numpy as np 
import pandas as pd 
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

# We dont Probably need the Gridlines. Do we? If yes comment this line
sns.set(style="ticks")

flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
flatui = sns.color_palette(flatui)


In [82]:
import scipy.stats as ss
from collections import Counter
import math 
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats


## Reading the Data and Preprocessing

In [83]:
def preprocess_data(X, preprocessor=None):
    if preprocessor is not None:
        preprocessor.fit(X)
        X_preprocessed = X.copy()
        X_preprocessed[:] = preprocessor.transform(X.values)
        X = X_preprocessed
    return X

In [84]:
from sklearn.preprocessing import Normalizer, StandardScaler

In [85]:
standarizer = StandardScaler()

In [86]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd 
df=pd.read_csv('gdrive/My Drive/DS Bootcamp 14-15.05/Zaawansowane problemy klasyfikacji/dataset/santander/train.csv')
df.pop('ID_code')
print(df.shape)
df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
(200000, 201)


,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [87]:
def get_feats(df): #dzieki temu nie będę brał niepotrzebnych kolumn do modelowania
    feats = [f for f in df.columns if f not in ['ID_code','target']]
    return feats

def get_X(df): #do pobierania macierzy X czyli cech które będę starał się opisywac modelem
    return df[ get_feats(df) ].values

def get_y(df, target_var='target'): #wektor y - informacja o predycji. w przypadku przykładowych danych to ":target"
    return df[target_var].values

In [88]:
df.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [89]:
y = df['target']
X = df.copy()
del X['target']
X = preprocess_data(X, standarizer)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [90]:
len(X.columns)

200

# Set some Parameters

In [91]:
feature_name = list(X.columns)
# no of maximum features we need to select
num_feats=30

# 1. Pearson correlation

This is a filter-based method. 
We check the absolute value of the Pearson's correlation between the target and numerical features in our dataset. We keep the top n features based on this criterion.

In [92]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


In [93]:
cor_feature

['var_44',
 'var_40',
 'var_109',
 'var_179',
 'var_115',
 'var_1',
 'var_0',
 'var_34',
 'var_198',
 'var_133',
 'var_148',
 'var_13',
 'var_165',
 'var_2',
 'var_190',
 'var_80',
 'var_166',
 'var_99',
 'var_21',
 'var_22',
 'var_174',
 'var_76',
 'var_26',
 'var_53',
 'var_146',
 'var_110',
 'var_6',
 'var_12',
 'var_139',
 'var_81']

# 2. Chi-Square Features

This is another filter-based method. 
In this method, we calculate the chi-square metric between the target and the numerical variable and only select the variable with the maximum chi-squared values.

In [94]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

30 selected features


In [95]:
chi_feature

['var_0',
 'var_1',
 'var_2',
 'var_6',
 'var_9',
 'var_12',
 'var_13',
 'var_21',
 'var_22',
 'var_26',
 'var_34',
 'var_40',
 'var_53',
 'var_76',
 'var_80',
 'var_81',
 'var_94',
 'var_99',
 'var_108',
 'var_109',
 'var_110',
 'var_139',
 'var_146',
 'var_148',
 'var_154',
 'var_165',
 'var_166',
 'var_174',
 'var_190',
 'var_198']

# 3. Recursive Feature Elimination

This is a wrapper based method. As I said before, wrapper methods consider the selection of a set of features as a search problem. 
From sklearn Documentation:

>>The goal of recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

As you would have guessed we could use any estimator with the method. In this case, we use LogisticRegression and the RFE observes the coef_ attribute of the LogisticRegression object

In [96]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
rfe_selector.fit(X_norm, y)

Fitting estimator with 200 features.
Fitting estimator with 190 features.
Fitting estimator with 180 features.
Fitting estimator with 170 features.
Fitting estimator with 160 features.
Fitting estimator with 150 features.
Fitting estimator with 140 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.


RFE(estimator=LogisticRegression(), n_features_to_select=30, step=10, verbose=5)

In [97]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

30 selected features


In [98]:
rfe_feature

['var_2',
 'var_6',
 'var_12',
 'var_21',
 'var_22',
 'var_26',
 'var_34',
 'var_44',
 'var_53',
 'var_67',
 'var_76',
 'var_80',
 'var_81',
 'var_99',
 'var_110',
 'var_115',
 'var_133',
 'var_139',
 'var_146',
 'var_148',
 'var_149',
 'var_165',
 'var_166',
 'var_169',
 'var_170',
 'var_174',
 'var_179',
 'var_190',
 'var_192',
 'var_198']

# 4. Lasso: SelectFromModel

This is an Embedded method. As said before, Embedded methods use algorithms that have built-in feature selection methods. 
For example, Lasso, and RF have their own feature selection methods. Lasso Regularizer forces a lot of feature weights to be zero. 
Here we use Lasso to select variables.

In [99]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
embeded_lr_selector.fit(X_norm, y)

SelectFromModel(estimator=LogisticRegression(), max_features=30)

In [100]:
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

30 selected features


In [101]:
embeded_lr_feature

['var_2',
 'var_6',
 'var_12',
 'var_21',
 'var_22',
 'var_26',
 'var_34',
 'var_44',
 'var_53',
 'var_67',
 'var_76',
 'var_80',
 'var_81',
 'var_99',
 'var_110',
 'var_115',
 'var_133',
 'var_139',
 'var_146',
 'var_148',
 'var_149',
 'var_165',
 'var_166',
 'var_169',
 'var_170',
 'var_174',
 'var_179',
 'var_190',
 'var_192',
 'var_198']

# 5. Tree-based: SelectFromModel

This is an Embedded method. As said before, Embedded methods use algorithms that have built-in feature selection methods.
We can also use RandomForest to select features based on feature importance.
We calculate feature importance using node impurities in each decision tree. In Random forest, the final feature importance is the average of all decision tree feature importance.

In [54]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
embeded_rf_selector.fit(X, y)

ValueError: ignored

In [ ]:
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

In [ ]:
embeded_rf_feature

We could also have used a LightGBM. Or an XGBoost object as long it has a feature_importances_ attribute.

In [ ]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
embeded_lgb_selector.fit(X, y)

In [ ]:
embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

In [ ]:
embeded_lgb_feature

# BONUS

In [102]:

pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support})#,
                                    #'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


,Feature,Pearson,Chi-2,RFE,Logistics,Total
1,var_99,True,True,True,True,4
2,var_81,True,True,True,True,4
3,var_80,True,True,True,True,4
4,var_76,True,True,True,True,4
5,var_6,True,True,True,True,4
6,var_53,True,True,True,True,4
7,var_34,True,True,True,True,4
8,var_26,True,True,True,True,4
9,var_22,True,True,True,True,4
10,var_21,True,True,True,True,4


Z.7. WYkonaj analogiczną prace na zbiorze santander tj. wczytaj go, przeskaluj (StandardScaler lub MinMaxScaler), użyj wszystki powyzszych metod, wybierz 15 najistotniejszych kolumn, uczenie wykonaj z GridSearchCV stworzonym w poprzednim ćwiczeniu oraz algorytmem XGBoost rowniez stworzonym w cwiczeniu poprzednim. 

*   PO wybraniu najistniojeszych cech wykonaj dla nich proces uczenia xgboostem oraz dla wskazanych przez GridSearchCV Parametrów. Jako rezultat zapisz f1_score na zb treningowym i testowym.




***Random Search***

In [103]:
columns = feature_selection_df[feature_selection_df['Total']==max(feature_selection_df['Total'])]['Feature'].tolist()
columns = feature_selection_df['Feature'][:30].tolist()
columns

['var_99',
 'var_81',
 'var_80',
 'var_76',
 'var_6',
 'var_53',
 'var_34',
 'var_26',
 'var_22',
 'var_21',
 'var_2',
 'var_198',
 'var_190',
 'var_174',
 'var_166',
 'var_165',
 'var_148',
 'var_146',
 'var_139',
 'var_12',
 'var_110',
 'var_44',
 'var_179',
 'var_133',
 'var_115',
 'var_67',
 'var_40',
 'var_192',
 'var_170',
 'var_169']

In [104]:
X=X[columns]
X.head()

,var_99,var_81,var_80,var_76,var_6,var_53,var_34,var_26,var_22,var_21,...,var_110,var_44,var_179,var_133,var_115,var_67,var_40,var_192,var_170,var_169
0,-1.431860,-0.383475,1.055809,-1.057511,-0.334926,-1.091818,-0.447275,-0.184673,-0.606166,-0.126734,...,-0.910467,0.456291,-0.311275,-0.654603,-0.268672,1.424133,0.179582,0.817469,-1.075697,-0.400904
1,0.773241,1.504277,-0.436865,-0.350196,0.244461,0.889088,-0.118257,-1.302434,1.491237,-1.771135,...,0.289561,-1.305067,3.031050,0.633727,0.741151,-0.544300,-0.530145,0.443623,1.252712,0.443627
2,0.386244,0.461860,0.225618,-0.194760,1.769839,0.519556,-0.718871,-0.665103,-1.085317,0.107340,...,-0.597095,-0.821217,-0.067934,-0.506308,0.263434,-0.283326,-0.820353,-0.174131,-1.601890,0.359986
3,1.446567,-1.817140,-0.673838,0.710052,0.500633,-1.045397,-1.523135,0.822099,0.883010,-0.573410,...,-0.321857,-0.152428,1.109222,1.017377,-1.815369,1.584647,0.349803,-0.818468,-1.615767,-0.793123
4,0.761214,-0.603299,-0.381249,-0.632825,0.613372,-0.330917,1.793625,1.220617,2.038227,0.207760,...,-1.492695,-0.345514,0.922795,-0.521750,-2.222564,-1.401858,0.136390,-1.405987,0.328676,-0.227938


In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2019)

In [106]:
from sklearn.model_selection import RandomizedSearchCV

In [107]:
random_param = {
    'max_depth': range(1,9),
    'learning_rate': np.logspace(np.log10(0.005), np.log10(0.5), base = 10, num = 1000),
    'n_estimators': range(100, 1000, 50),
    'gamma': np.linspace(0, 1, 10),
    'min_child_weight': range(1,100,5),
    'subsample': np.linspace(0.5, 1, 101),
    'colsample_bytree': np.linspace(0.5, 1, 101),
    'colsample_bylevel': np.linspace(0.5, 1, 101),
    'reg_alpha': np.linspace(0, 2),
    'reg_lambda': np.linspace(0, 2),
    'tree_method': ['gpu_hist']
}

In [108]:
import xgboost as xgb

In [109]:
model = xgb.XGBClassifier()

random_search = RandomizedSearchCV(estimator=model
                                 , param_distributions=random_param
                                 , scoring='accuracy' )

In [110]:
random_search.fit(X=X_train, y=y_train)

best_parameters = random_search.best_params_  
print(best_parameters) 

{'tree_method': 'gpu_hist', 'subsample': 0.555, 'reg_lambda': 1.2653061224489794, 'reg_alpha': 1.6326530612244896, 'n_estimators': 750, 'min_child_weight': 76, 'max_depth': 7, 'learning_rate': 0.03579520542982444, 'gamma': 0.1111111111111111, 'colsample_bytree': 0.9299999999999999, 'colsample_bylevel': 0.585}


In [111]:
model = xgb.XGBClassifier(**best_parameters)
model.fit(X_train, y_train)  
y_xgb = model.predict(X_test) 
from sklearn.metrics import f1_score
f1_score(y_test, y_xgb)

0.24553748512495038

In [112]:
RS_results = pd.DataFrame(random_search.cv_results_)
RS_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_tree_method,param_subsample,param_reg_lambda,param_reg_alpha,param_n_estimators,param_min_child_weight,...,param_colsample_bylevel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.291907,0.016668,0.016863,0.000879,gpu_hist,0.865,0.040816,0.816327,250,36,...,0.605,"{'tree_method': 'gpu_hist', 'subsample': 0.865...",0.899969,0.899969,0.899937,0.899937,0.899937,0.899950,0.000015,8
1,3.121106,0.003627,0.018433,0.001435,gpu_hist,0.64,0.979592,1.55102,600,91,...,0.94,"{'tree_method': 'gpu_hist', 'subsample': 0.64,...",0.903906,0.904406,0.905219,0.904875,0.904687,0.904619,0.000443,4
2,4.525055,0.061813,0.021675,0.002906,gpu_hist,0.94,0.244898,0.122449,600,51,...,0.875,"{'tree_method': 'gpu_hist', 'subsample': 0.94,...",0.903813,0.903781,0.904375,0.904563,0.903750,0.904056,0.000343,5
3,2.495045,0.003476,0.017293,0.000914,gpu_hist,0.995,1.387755,0.44898,500,66,...,0.95,"{'tree_method': 'gpu_hist', 'subsample': 0.995...",0.904125,0.904656,0.905344,0.905438,0.905094,0.904931,0.000485,3
4,3.496350,0.001656,0.018779,0.000169,gpu_hist,0.56,1.265306,0.938776,500,91,...,0.83,"{'tree_method': 'gpu_hist', 'subsample': 0.56,...",0.905344,0.905781,0.907438,0.906031,0.906031,0.906125,0.000703,2


***Grid Search ***

In [113]:
grid_param = {  
    'n_estimators': [50, 60, 70 ,80],
    'max_depth': [5,6,7,8],
    'learning_rate': [ 0.1, 0.25,0.5, 1],
    'subsample': [0.75, 1.00], 
    'tree_method': ['gpu_hist']    
}

In [114]:
from sklearn.model_selection import GridSearchCV

In [115]:
model = xgb.XGBClassifier()

grid_search = GridSearchCV(estimator=model,  
                     param_grid=grid_param,
                     scoring='accuracy')

In [116]:
random_search.fit(X=X_train, y=y_train)

best_parameters = random_search.best_params_  
print(best_parameters) 

{'tree_method': 'gpu_hist', 'subsample': 0.56, 'reg_lambda': 1.7142857142857142, 'reg_alpha': 1.4693877551020407, 'n_estimators': 450, 'min_child_weight': 26, 'max_depth': 5, 'learning_rate': 0.03889205535643245, 'gamma': 0.0, 'colsample_bytree': 0.94, 'colsample_bylevel': 0.595}


In [117]:
model = xgb.XGBClassifier(**best_parameters)
model.fit(X_train, y_train)
y_xgb = model.predict(X_test) 
from sklearn.metrics import f1_score
f1_score(y_test, y_xgb)

0.22844390342868737

In [119]:
f1_score(y_train, model.predict(X_train))

0.2834983161547097